In [1]:
%run pulsar_stars.ipynb

In [2]:
def binary_classification_exec(epoch_count=10, mb_size=10, report=1,train_rate = 0.8, adjust_ratio = False):
    binary_load_dataset(adjust_ratio)
    init_model()
    train_and_test(epoch_count, mb_size, report, train_rate)

In [3]:
def binary_load_dataset(adjust_ratio):
    pulsars, stars = [],[]
    with open('../data/pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        
        for row in csvreader:
          if row[8] == '1' : pulsars.append(row)
          else:
            stars.append(row)            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8, 1
    star_cnt, pulsar_cnt = len(stars), len(pulsars)
    if adjust_ratio:
      data = np.zeros([2*star_cnt, 9])
      data[0:star_cnt, :] = np.asarray(stars, dtype = 'float32')

      for n in range(star_cnt):
        data[star_cnt + n] = np.asarray(pulsars[n % pulsar_cnt], dtype = 'float32')

    else:
      data = np.zeros([star_cnt+pulsar_cnt, 9])
      data[0:star_cnt, :] = np.asarray(stars, dtype = 'float32')
      data[star_cnt:, :] = np.asarray(pulsars, dtype = 'float32')     

In [4]:
def forward_postproc(output, y):
    entropy = sigmoid_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy)
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)    
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

In [5]:
def eval_accuracy(output, y):
    est_yes = np.greater(output, 0)
    ans_yes = np.greater(y, 0.5)

    est_no = np.logical_not(est_yes)
    ans_no = np.logical_not(ans_yes)
    
    tp = np.sum(np.logical_and(est_yes, ans_yes))
    fp = np.sum(np.logical_and(est_yes, ans_no))
    fn = np.sum(np.logical_and(est_no, ans_no))
    tn = np.sum(np.logical_and(est_no, ans_yes))
    
    accuracy = safe_div(tp+tn, tp+tn+fp+fn)
    precision = safe_div(tp, tp+fp)
    recall = safe_div(tp, tp+fn)
    f1 = 2 * safe_div(recall*precision, recall+precision)
    
    return [accuracy, precision, recall, f1]

def safe_div(p, q):
    p, q = float(p), float(q)
    if np.abs(q) < 1.0e-20: return np.sign(p)
    return p / q

In [6]:
def train_and_test(epoch_count, mb_size, report, train_rate):
    step_count = arrange_data(mb_size, train_rate)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, \n accuracy:{:5.3f}, precision:{:5.3f}, recall:{:5.3f}, f1:{:5.3f}'. \
                  format(epoch+1, np.mean(losses), acc[0],acc[1],acc[2],acc[3]))
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: accuracy:{:5.3f}, precision:{:5.3f}, recall:{:5.3f}, f1:{:5.3f}'.\
          format(final_acc[0],final_acc[1], final_acc[2], final_acc[3]))

In [7]:
def relu(x):
    return np.maximum(x, 0)

def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))
        
def sigmoid_derv(x, y):
    return y * (1 - y)

def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)


